In [ ]:
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import os
from dotenv import load_dotenv

load_dotenv()

# Enables `response_model`
client = instructor.patch(OpenAI())

In [ ]:


class UserDetail(BaseModel):
    name: str
    age: int

user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Brian is 46 years old"},
    ]
)

assert isinstance(user, UserDetail)
assert user.name == "Brian"
assert user.age == 46

json_str = user.model_dump_json()
print(json_str)

In [ ]:

class Website(BaseModel):
    name: str
    url: str

class WebsiteExtraction(BaseModel):
    websites: List[Website] = Field(
        ...,
        description="Body of the answer, each website should be a separate object with a name and a url of the website",
    )

def get_websites(prompt) -> WebsiteExtraction:
    website_extraction: WebsiteExtraction = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=WebsiteExtraction,
        messages=[
        {
            "role": "system",
            "content": "You are an expert aws cloud solutions architect with 20 years experience. answer exactly what the question asks using the context.",
        },            
            {"role": "user", "content": prompt},
        ]
    )
    return '\n'.join([f"{website.name}, {website.url}" for website in website_extraction.websites])
    

print(get_websites("Extract the top 20 technology websites"), "\n")

print(get_websites("""
    Extract the top 20 AWS twitter accounts.
    Each twitter account contain a name and url.
    The twitter account should not be owned by AWS.
"""), "\n")

print(get_websites("""
    Extract the top 20 AWS websites.
    Each website should contain a name and url
    The website should not be owned by AWS .
    The website url should not have 'aws.amazon.com' as a substring
"""), "\n")


In [56]:
class File(BaseModel):
    """
    Correctly named file with contents.
    """

    file_name: str = Field(
        ..., description="The name of the file including the extension"
    )
    body: str = Field(..., description="Correct contents of a file")

    def save(self):
        with open(self.file_name, "w") as f:
            f.write(self.body)


class Program(BaseModel):
    """
    Set of files that represent a complete and correct program
    """

    files: List[File] = Field(..., description="List of files")


def develop(data: str) -> Program:
    return client.chat.completions.create(
        #model="gpt-3.5-turbo-0613",
        model='gpt-4-1106-preview',
        temperature=0.1,
        response_model=Program,
        messages=[
            {
                "role": "system",
                "content": "You are a world class programming AI capable of writing correct python scripts and modules. You will name files correct, include __init__.py files and write correct python code with correct imports.",
            },
            {
                "role": "user",
                "content": data,
            },
        ],
        max_tokens=1000,
    )

def codegen(prompt, output_directory_path):
    debug_output = ''
    os.makedirs(output_directory_path, exist_ok=True)
    program = develop(prompt)
    for file in program.files:
        open(f"{output_directory_path}/{file.file_name}", "w").write(file.body)
        debug_output += file.file_name
        debug_output += "\n-\n"
        debug_output += file.body
        debug_output += "\n\n\n"
    return debug_output


# program = develop(
#         """
#         Create a fastapi app with a readme.md file and a main.py file with
#         some basic math functions. the datamodels should use pydantic and
#         the main.py should use fastapi. the readme.md should have a title
#         and a description. The readme should contain some helpful infromation
#         and a curl example"""
#     )

# program = develop(
#         """
#         Create a command line program with a readme.md file and a main.py file
#         that will take an absolute directory path as a command line argument and recursively print
#         the files and directories in the directory.  The  the datamodels should use pydantic and
#         the main.py should use argparse. the readme.md should have a title
#         and a description. The readme should contain some helpful infromation
#         and a command line usage example"""
#     )



# program_debug_output = codegen(
#         """
#         Create a command line program with a readme.md file and a main.py file
#         that will take an absolute directory path as a command line argument and recursively print
#         the files and directories in the directory.  The  the datamodels should use pydantic and
#         the main.py should use argparse. the readme.md should have a title
#         and a description. The readme should contain some helpful infromation
#         and a command line usage example""",
#         "./codegen-recursive-directory-print"
#     )

program_debug_output = codegen(
        """
        Create a command line program with a readme.md file and a main.py file
        that will take a url and output filename as arguments.  It should download the url contents and save as filename.
        It should use argparse. The readme.md should have a title.
        It should include a requirements.txt file containing the program dependencies.
        The readme should contain some helpful information, commands to create virtual environment for the program to run
        and a command line usage example""",
        "./codegen-url-download"
    )

print(program_debug_output)

README.md
-
# URL Downloader

This command line program allows you to download the contents of a specified URL and save it to a file on your local system.

## Requirements

Before running the script, ensure you have Python installed on your system and the required packages installed using:

```
pip install -r requirements.txt
```

## Usage

To use the program, run the following command in your terminal:

```
python main.py --url <URL> --output <FILENAME>
```

Replace `<URL>` with the URL you want to download from and `<FILENAME>` with the desired output file name.

## Example

```
python main.py --url http://example.com/data.txt --output downloaded_data.txt
```

This will download the content from `http://example.com/data.txt` and save it as `downloaded_data.txt` in the current directory.


main.py
-
import argparse
import requests

# Set up the argument parser
parser = argparse.ArgumentParser(description='Download content from a URL and save it to a file.')
parser.add_argument('--url'